In [1]:
from typing import Annotated
import operator,json
from typing import TypedDict, Annotated, Sequence
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph,END,START
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults

In [2]:
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="Gemma2-9b-It")

In [3]:
@tool
def multiply(first_number:int, second_number:int)->int:
    """multiply two integer number"""
    return first_number * second_number


In [4]:
multiply({"first_number":24,"second_number":364})

C:\Users\LapMaster\AppData\Local\Temp\ipykernel_26492\2089873997.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  multiply({"first_number":24,"second_number":364})


8736

In [5]:
multiply.invoke({"first_number":24,"second_number":364})

8736

In [6]:
@tool
def search(query:str):
    """perform the web search on the user query"""
    tavily=TavilySearchResults()
    result=tavily.invoke(query)
    return result

In [7]:
search.invoke("who is a current president of USA?")


[{'title': 'Presidents, vice presidents, and first ladies | USAGov',
  'url': 'https://www.usa.gov/presidents',
  'content': "Learn about the duties of president, vice president, and first lady of the United States. Find out how to contact and learn more about current and past leaders.\n\nPresident of the United States\n\nThe president of the United States is the:\n\nCurrent president\n\nThe 47th and current president of the United States is Donald John Trump.\xa0He was sworn into office on January 20, 2025.\n\nFormer U.S. presidents [...] Read about past presidents and vice presidents.\n\nFirst lady\n\nThe First lady of the United States has traditionally been the wife or other close female relative of the president of the United States. First ladies:\n\nCurrent first lady\n\nThe current first lady of the United States is Melania Trump.\n\nFormer first ladies\n\nFind a list of former first ladies and their presidential spouses.\n\nSee the Smithsonian Institution's virtual tour of firs

In [8]:
tools=[search,multiply]

In [9]:
model_with_tools = llm.bind_tools(tools)


In [10]:
tool_mapping={tool.name: tool for tool in tools}

In [11]:
tool_mapping

{'search': StructuredTool(name='search', description='perform the web search on the user query', args_schema=<class 'langchain_core.utils.pydantic.search'>, func=<function search at 0x0000024E38439AB0>),
 'multiply': StructuredTool(name='multiply', description='multiply two integer number', args_schema=<class 'langchain_core.utils.pydantic.multiply'>, func=<function multiply at 0x0000024E38438E50>)}

In [12]:
response = model_with_tools.invoke("who is a current president of USA?")


In [13]:
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_y9c3', 'function': {'arguments': '{"query":"current president of USA"}', 'name': 'search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 1076, 'total_tokens': 1158, 'completion_time': 0.149090909, 'prompt_time': 0.038092135, 'queue_time': 0.194725657, 'total_time': 0.187183044}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--096cb4a1-0ed5-4fbd-9b02-3101bb410c81-0', tool_calls=[{'name': 'search', 'args': {'query': 'current president of USA'}, 'id': 'call_y9c3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1076, 'output_tokens': 82, 'total_tokens': 1158})

In [14]:
response.additional_kwargs.get("tool_calls")


[{'id': 'call_y9c3',
  'function': {'arguments': '{"query":"current president of USA"}',
   'name': 'search'},
  'type': 'function'}]

In [15]:
response.additional_kwargs.get("tool_calls")[0]["function"]["name"]

'search'

In [16]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [17]:
def invoke_model(state:AgentState):
    messages = state['messages']
    question = messages[-1]   ## Fetching the user question
    return {"messages":[model_with_tools.invoke(question)]}

In [18]:
def invoke_tool(state:AgentState):
    tool_details= state['messages'][-1].additional_kwargs.get("tool_calls", [])[0]
    
    if tool_details is None:
        raise Exception("no tool call found")
    
    print(f'Selected tool: {tool_details.get("function").get("name")}')
    
    if tool_details.get("function").get("name")=="search":
        response = input(prompt=f"[y/n] continue with expensive web search?")
        if response == "n":
            raise Exception("web search discard")
        
    response = tool_mapping[tool_details['function']['name']].invoke(json.loads(tool_details.get("function").get("arguments")))
    return {"messages" : [response]}

In [19]:
def router(state):
    tool_calls = state['messages'][-1].additional_kwargs.get("tool_calls", [])
    if len(tool_calls):
        return "tool"
    else:
        return "end"

In [20]:
graph = StateGraph(AgentState) ### StateGraph with AgentState

graph.add_node("ai_assistant", invoke_model)

graph.add_node("tool", invoke_tool)

In [21]:
graph.add_conditional_edges("ai_assistant", router, {"tool": "tool","end": END,})

graph.add_edge("tool", END)

#graph.add_edge("tool", "ai_assistant")

graph.set_entry_point("ai_assistant")

In [22]:
app = graph.compile()

In [45]:

for s in app.stream({"messages": ["who is upcoming president of USA?"]}):
    print(list(s.values())[0])
    print("----")
     

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_q38r', 'function': {'arguments': '{"query":"upcoming president of USA"}', 'name': 'search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 83, 'prompt_tokens': 1075, 'total_tokens': 1158, 'completion_time': 0.150909091, 'prompt_time': 0.049138961, 'queue_time': 0.171547687, 'total_time': 0.200048052}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--4cedc64f-21d2-443e-9ac8-725f93934989-0', tool_calls=[{'name': 'search', 'args': {'query': 'upcoming president of USA'}, 'id': 'call_q38r', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1075, 'output_tokens': 83, 'total_tokens': 1158})]}
----
Selected tool: search
{'messages': [[{'title': '2024 United States presidential election - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/2024_United_States_presidential_election', 'content': "A pres

In [46]:

for s in app.stream({"messages": ["what is multiplication of 23 and 46?"]}):
    print(list(s.values())[0])
    print("----")

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1htr', 'function': {'arguments': '{"first_number":23,"second_number":46}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 92, 'prompt_tokens': 1080, 'total_tokens': 1172, 'completion_time': 0.167272727, 'prompt_time': 0.037871462, 'queue_time': 0.196639479, 'total_time': 0.205144189}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--7ab76e54-f84a-4d49-8538-37d64df895fa-0', tool_calls=[{'name': 'multiply', 'args': {'first_number': 23, 'second_number': 46}, 'id': 'call_1htr', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1080, 'output_tokens': 92, 'total_tokens': 1172})]}
----
Selected tool: multiply
{'messages': [1058]}
----


In [47]:

for s in app.stream({"messages": ["what is the total amount of money exist over the earth?"]}):
    print(list(s.values())[0])
    print("----")

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_6mdm', 'function': {'arguments': '{"query":"what is the total amount of money exist over the earth"}', 'name': 'search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 1080, 'total_tokens': 1170, 'completion_time': 0.163636364, 'prompt_time': 0.037873092, 'queue_time': 0.166096862, 'total_time': 0.201509456}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--b9bf97a4-1b6a-48ad-b9ba-5461c48712c4-0', tool_calls=[{'name': 'search', 'args': {'query': 'what is the total amount of money exist over the earth'}, 'id': 'call_6mdm', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1080, 'output_tokens': 90, 'total_tokens': 1170})]}
----
Selected tool: search
{'messages': [[{'title': 'How Much Money Exists on Planet Earth? | Celebrity Net Worth', 'url': 'https://www.celebritynetw

In [48]:
#LangGraph supports human-in-the-loop workflows in a number of ways. In this section, we will use LangGraph's interrupt_before functionality to always break the tool node.

In [51]:
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]

In [52]:
tavily=TavilySearchResults()

In [53]:
tools = [tavily]

In [54]:
llm_with_tools = llm.bind_tools(tools)


In [55]:
def ai_assistant(state: AgentState):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

In [56]:
memory = MemorySaver()

In [57]:
graph_builder = StateGraph(AgentState)
graph_builder.add_node("ai_assistant", ai_assistant)

tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

In [58]:
graph_builder.add_edge(START, "ai_assistant")

graph_builder.add_conditional_edges(
    "ai_assistant",
    tools_condition,
)
graph_builder.add_edge("tools", "ai_assistant")

In [59]:
app2 = graph_builder.compile(
    checkpointer=memory,
    # This is new!
    interrupt_before=["tools"],
    # Note: can also interrupt __after__ tools, if desired.
    # interrupt_after=["tools"]
)

In [61]:

user_input = "what is current a capital of india?"
config = {"configurable": {"thread_id": "1"}}

In [62]:

# The config is the **second positional argument** to stream() or invoke()!
events = app2.stream(
    {"messages": [("user", user_input)]}, config, stream_mode="values"
)

In [63]:
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

what is current a capital of india?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_1gjr)
 Call ID: call_1gjr
  Args:
    query: What is the current capital of India?


In [68]:

# `None` will append nothing new to the current state, letting it resume as if it had never been interrupted
events = app2.stream(None, config, stream_mode="values")

In [69]:
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_as7q)
 Call ID: call_as7q
  Args:
    query: What is the capital of India?
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "What is the capital of India? States and union territories explained.", "url": "https://www.usatoday.com/story/news/world/2023/05/24/what-is-the-capital-of-india/70195720007/", "content": "Want to learn more about the soon-to-be most populous country? Here’s some interesting information about how India is organized.\n\nWhat is the capital of India?\n\nThe capital of India is New Delhi, located in the north-central part of the country to the west of the Yamuna River.\n\nCalcutta (now Kolkata, the capital of West Bengal) was the country’s capital until 1911 when King George V declared Delhi the new capital and construction of New Delhi began. [...] When the nation